In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client pandas numpy scikit-learn

In [ ]:
import json
import pandas as pd
import numpy as np
from google.oauth2 import service_account
from googleapiclient.discovery import build
from sklearn.metrics.pairwise import cosine_similarity

# 서비스 계정 키 파일 경로
KEY_FILE_LOCATION = 'path/to/your/service-account-key.json'
VIEW_ID = 'YOUR_VIEW_ID'

# Google Analytics API 초기화
def initialize_analyticsreporting():
    credentials = service_account.Credentials.from_service_account_file(
        KEY_FILE_LOCATION,
        scopes=['https://www.googleapis.com/auth/analytics.readonly']
    )
    analytics = build('analyticsreporting', 'v4', credentials=credentials)
    return analytics

# 모든 사용자 데이터 요청 함수
def get_all_users_report(analytics):
    return analytics.reports().batchGet(
        body={
            'reportRequests': [
                {
                    'viewId': VIEW_ID,
                    'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'today'}],
                    'metrics': [{'expression': 'ga:avgTimeOnPage'}],
                    'dimensions': [{'name': 'ga:pagePath'}, {'name': 'ga:clientId'}],  # clientId를 사용자 ID로 사용
                }]
        }
    ).execute()

# 응답에서 데이터 추출
def process_response(response):
    data = []
    for report in response.get('reports', []):
        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

        for row in report.get('data', {}).get('rows', []):
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            row_data = {}
            for header, dimension in zip(dimensionHeaders, dimensions):
                row_data[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metricHeader, value in zip(metricHeaders, values.get('values')):
                    row_data[metricHeader.get('name')] = float(value)

            data.append(row_data)

    return pd.DataFrame(data)

# 콘텐츠 기반 필터링을 위한 코사인 유사도 계산
def calculate_cosine_similarity(data, target_page, feature_columns, weights):
    # 가중치를 적용하여 특징 벡터 생성
    weighted_data = data[feature_columns] * weights
    target_features = weighted_data[data['ga:pagePath'] == target_page]
    similarities = cosine_similarity(weighted_data, target_features)
    data['similarity'] = similarities[:, 0]
    return data

# 주요 로직
def main():
    analytics = initialize_analyticsreporting()
    response = get_all_users_report(analytics)
    df = process_response(response)

    # 사용자의 ID 별로 체류 시간이 가장 긴 페이지를 찾습니다.
    top_pages = df.loc[df.groupby('ga:clientId')['ga:avgTimeOnPage'].idxmax()]

    # 예시 데이터: 각 페이지에 대한 특징 (카테고리, 활동요일, 회비 등)
    features = pd.DataFrame({
        'ga:pagePath': ['/page1', '/page2', '/page3'],
        'activity': [1, 2, 3],  # 예시 특징
        'day': [1, 2, 3],
        'fee': [10, 20, 30],
        'interest': [5, 10, 15]
    })

    # 가중치 설정
    weights = np.array([1, 1, 1, 1])  # 가중치를 설정할 수 있습니다.

    # 체류 시간이 가장 긴 페이지를 기준으로 유사한 페이지 추천
    for user, page in zip(top_pages['ga:clientId'], top_pages['ga:pagePath']):
        print(f"User {user} spent the most time on {page}")
        similar_pages = calculate_cosine_similarity(features, page, ['activity', 'day', 'fee', 'interest'], weights)
        recommendations = similar_pages.sort_values(by='similarity', ascending=False)
        print(f"Recommendations for user {user} based on page {page}:")
        print(recommendations[['ga:pagePath', 'similarity']])

if __name__ == '__main__':
    top_n = 5  # 추천받을 개수를 설정합니다.
    main(top_n)


#Google Analytics로 체류 시간 수집 및 분석하기


###1.Google Analytics 계정 설정

Google Analytics에 로그인하고 새 속성을 생성합니다.
웹사이트 또는 플랫폼의 추적 ID를 생성하고, 해당 추적 ID를 웹사이트나 플랫폼의 추적 코드에 추가합니다.


###2.사용자 및 이벤트 추적 설정

페이지 체류 시간을 추적하려면, Google Analytics는 기본적으로 각 페이지의 평균 체류 시간을 자동으로 수집합니다.
필요한 경우, gtag.js 또는 analytics.js를 사용하여 추가 이벤트를 설정할 수 있습니다.


###3.체류 시간 데이터 접근

Google Analytics 대시보드에서 행동 → 사이트 콘텐츠 → 모든 페이지에서 페이지별 체류 시간 데이터를 확인할 수 있습니다.
보고서에서 페이지의 평균 체류 시간(ga
)과 같은 메트릭을 추출할 수 있습니다.


###4.Google Analytics Reporting API 사용

무료 Google Analytics API를 사용하여 프로그래밍적으로 데이터를 쿼리하고, 체류 시간 데이터를 자동으로 가져올 수 있습니다.
Google Analytics Reporting API 문서를 참고하여 체류 시간 데이터를 수집하는 코드를 작성할 수 있습니다.


###5.데이터 처리 및 분석

수집된 데이터를 기반으로 Pandas, NumPy, Scikit-Learn 등 Python 라이브러리를 사용하여 데이터를 처리하고, 추천 시스템을 구현할 수 있습니다.